# Business Level Processing
BY EDISON LIM JUN HAO

Second level of HOG analysis 
For each business, perform feature pooling (or stacking) using averages of the features. 


In [1]:
import numpy as np 
import pandas as pd 
import h5py
import time

DATA = '/Users/edison/Desktop/Yelp/project/'

In [2]:
#photo to business mapping 
train_photo_to_business = pd.read_csv(DATA + 'train_photo_to_biz_ids.csv')
#train labels 
train_labels = pd.read_csv(DATA + 'train.csv').dropna()

In [15]:
train_photo_to_business.head()

,photo_id,business_id
0,204149,3034
1,52779,2805
2,278973,485
3,195284,485
4,19992,485


In [16]:
train_labels.head()

,business_id,labels
0,1000,1 2 3 4 5 6 7
1,1001,0 1 6 8
2,100,1 2 4 5 6 7
3,1006,1 2 4 5 6
4,1010,0 6 8


In [3]:
train_labels['labels'] = train_labels['labels'].apply(lambda x: tuple(sorted(int(t) for t in x.split())))
train_labels.set_index('business_id', inplace=True)

In [4]:
train_labels.head()

,labels
business_id,
1000,"(1, 2, 3, 4, 5, 6, 7)"
1001,"(0, 1, 6, 8)"
100,"(1, 2, 4, 5, 6, 7)"
1006,"(1, 2, 4, 5, 6)"
1010,"(0, 6, 8)"


In [5]:
len(train_labels)

1996

In [6]:
len(train_labels.index.unique())

1996

* Load image features from the HOG file

In [7]:
## Load image features
# Image features is a 2048 dimension
# 234842 * 2048 2d array 
f = h5py.File(DATA+'train_image_HOGfeatures.h5','r')
train_image_features = np.copy(f['feature'])
f.close()


In [13]:
train_image_features.shape

(234842, 2048)

In [14]:
train_image_features

array([[ 0.34695926,  0.07337682,  0.01276903, ...,  0.14800219,
         0.11983313,  0.17191491],
       [ 0.38935474,  0.07249777,  0.01099139, ...,  0.03001626,
         0.        ,  0.        ],
       [ 0.04052499,  0.00253592,  0.00152215, ...,  0.0923799 ,
         0.18500596,  0.14189687],
       ..., 
       [ 0.13786054,  0.12325875,  0.10942163, ...,  0.10116033,
         0.06392809,  0.19003363],
       [ 0.26071924,  0.14863959,  0.02482761, ...,  0.15613814,
         0.03136114,  0.02775179],
       [ 0.15591125,  0.14017394,  0.12439339, ...,  0.09448919,
         0.14516343,  0.07755874]], dtype=float32)

### Mapping IDs of business to photo
One business can have more than one photo. To cater for multi-photo problem, mean is used. 

However, there are other problems associated for it. 


In [3]:
#map the business ID to the correct photo ID 
df = pd.DataFrame(columns=['business', 'label', 'features'])
business_id = train_labels.index
index = 0 
for i in business_id: 
    label = train_labels.loc[i]['labels']
    image_index = train_photo_to_business[train_photo_to_business['business_id']==i].index.tolist()
    features = train_image_features[image_index]
    features = list(np.mean(features, axis=0))
    
    df.loc[index] = [i, label, features]
    index = index + 1 

NameError: name 'train_labels' is not defined

In [32]:
train_labels.loc[1000]['labels']

(1, 2, 3, 4, 5, 6, 7)

In [33]:
df.head()

,business,label,features
0,1000.0,"(1, 2, 3, 4, 5, 6, 7)","[0.139584, 0.0850379, 0.0901607, 0.125576, 0.1..."
1,1001.0,"(0, 1, 6, 8)","[0.120329, 0.0501645, 0.0770496, 0.115872, 0.1..."
2,100.0,"(1, 2, 4, 5, 6, 7)","[0.143955, 0.077672, 0.0878393, 0.13833, 0.195..."
3,1006.0,"(1, 2, 4, 5, 6)","[0.163665, 0.0782825, 0.121123, 0.0852653, 0.1..."
4,1010.0,"(0, 6, 8)","[0.174046, 0.0706035, 0.112249, 0.115906, 0.13..."


In [35]:
with open(DATA+"biz_label_features(train).csv",'w') as f:  
    df.to_csv(f, index=False)

## Testing Dataset

Note: I am not successful in my attempts to train the testing dataset. HOG analysis results are produced by splitting the train dataset (80/20)

I decided not to spend time rectifying this portion as I believe I should spend more time on CNN.

In [3]:
test_photo_to_biz = pd.read_csv(DATA+'test_photo_to_biz.csv')
test_photo_to_biz.head()

,photo_id,business_id
0,317818,003sg
1,30679,003sg
2,455084,003sg
3,371381,003sg
4,86224,003sg


In [4]:
test_photo_to_biz = pd.read_csv(DATA+'test_photo_to_biz.csv')
biz_ids = test_photo_to_biz['business_id'].unique()

## Load mapping

* What's wrong? For some unknown result, the function is unable to find some files within the directory. The file exists in the directory, so there must be something wrong with the lookup.

In [10]:
f = h5py.File(DATA+'test_image_HOGfeatures.h5','r')
image_filenames = list(np.copy(f['photo_id']))
image_filenames = [name.split('/')[-1][:-4] for name in image_filenames]  #remove the full path and the str ".jpg"
image_features = np.copy(f['feature'])
f.close()
print "Number of business: ", len(biz_ids)

df = pd.DataFrame(columns=['business','feature vector'])
index = 0
t = time.time()

for biz in biz_ids:     
    
    image_ids = test_photo_to_biz[test_photo_to_biz['business_id']==biz]['photo_id'].tolist()  
    image_index = [image_filenames.index(str(x)) for x in image_ids]
     
    folder = DATA+'test_photos/'            
    features = image_features[image_index]
    mean_feature =list(np.mean(features,axis=0))

    df.loc[index] = [biz, mean_feature]
    index+=1
    if index%1000==0:
        print "Business processed: ", index, "Time passed: ", "{0:.1f}".format(time.time()-t), "sec"

with open(data_root+"test_biz_HOGfeatures.csv",'w') as f:  
    df.to_csv(f, index=False)

Number of business:  10000


ValueError: '317818' is not in list

In [11]:
f = h5py.File(DATA+'test_image_HOGfeatures.h5','r')
image_filenames = list(np.copy(f['photo_id']))
image_filenames = [name.split('/')[-1][:-4] for name in image_filenames]  #remove the full path and the str ".jpg"
image_features = np.copy(f['feature'])
f.close()
print "Number of business: ", len(biz_ids)

Number of business:  10000


In [15]:
len(image_filenames)

1190225